In [1]:
!pip install git+https://github.com/eclipse-qrisp/Qrisp.git@catalyst_integration

  Cloning https://github.com/eclipse-qrisp/Qrisp.git (to revision catalyst_integration) to /tmp/pip-req-build-s03df8r3
  Running command git clone --filter=blob:none --quiet https://github.com/eclipse-qrisp/Qrisp.git /tmp/pip-req-build-s03df8r3
  error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
  error: 7516 bytes of body are still expected
  fetch-pack: unexpected disconnect while reading sideband packet
  fatal: early EOF
  fatal: fetch-pack: invalid index-pack output
^C
ERROR: Operation cancelled by user


Jasp, a dynamic Pythonic low-level IR
-------------------------------------

Within this notebook we demonstrate the latest feature of the Jax Integration.

We introduce a Jasp, a new IR that represents hybrid programs embedded into the Jaxpr IR.

Creating a Jasp program is simple:

In [2]:
from qrisp import *
from qrisp.jasp import *
from jax import make_jaxpr


def main(i):
    qf = QuantumFloat(i)
    h(qf[0])
    cx(qf[0], qf[1])

    meas_float = measure(qf)

    return meas_float
    

jaspr = make_jaspr(main)(5)

print(jaspr)

{ lambda ; a:QuantumCircuit b:i64[]. let
    c:QuantumCircuit d:QubitArray = jasp.create_qubits a b
    e:Qubit = jasp.get_qubit d 0
    f:QuantumCircuit = jasp.h c e
    g:Qubit = jasp.get_qubit d 1
    h:QuantumCircuit = jasp.cx f e g
    i:QuantumCircuit j:i64[] = jasp.measure h d
    k:QuantumCircuit = jasp.reset i d
    l:QuantumCircuit = jasp.delete_qubits k d
  in (l, j) }


Jasp programs can be executed with the Jasp interpreter by calling them like a function:

In [3]:
print(jaspr(5))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


0                                                                                    


A quicker way to do this is to use the ``jaspify`` decorator. This decorator automatically transforms the function into a Jaspr and calls the simulator

In [4]:
@jaspify
def main(i):
    qf = QuantumFloat(i)
    h(qf[0])
    cx(qf[0], qf[1])

    meas_float = measure(qf)

    return meas_float

print(main(5))

0                                                                                    


Jasp programs can be compiled to QIR, which is one of the most popular low-level representations for quantum computers. For that you need Catalyst installed (only on Mac & Linux).

In [5]:
try:
    import catalyst
except:
    !pip install pennylane-catalyst

In [6]:
qir_string = jaspr.to_qir()
print(qir_string[:2500])

; ModuleID = 'LLVMDialectModule'
source_filename = "LLVMDialectModule"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-i128:128-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@"{'shots': 0, 'mcmc': False, 'num_burnin': 0, 'kernel_name': None}" = internal constant [66 x i8] c"{'shots': 0, 'mcmc': False, 'num_burnin': 0, 'kernel_name': None}\00"
@LightningSimulator = internal constant [19 x i8] c"LightningSimulator\00"
@"/home/positr0nium/miniconda3/envs/qrisp/lib/python3.10/site-packages/catalyst/utils/../lib/librtd_lightning.so" = internal constant [111 x i8] c"/home/positr0nium/miniconda3/envs/qrisp/lib/python3.10/site-packages/catalyst/utils/../lib/librtd_lightning.so\00"

declare void @__catalyst__rt__finalize()

declare void @__catalyst__rt__initialize(ptr)

declare void @__catalyst__qis__PauliX(ptr, ptr)

declare ptr @__catalyst__qis__Measure(ptr, i32)

declare void @__catalyst__qis__CNOT(ptr, ptr, ptr)

declare void @__catalyst__qis__Hada

The Qache decorator
-------------------

One of the most powerful features of this IR is that it is fully dynamic, allowing many functions to be cached and reused. For this we have the ``qache`` decorator. Qached functions are only excutes ones (per calling signature) and otherwise retrieved from cache.

In [7]:
import time

@qache
def inner_function(qv, i):
    cx(qv[0], qv[1])
    h(qv[i])
    # Complicated compilation, that takes a lot of time
    time.sleep(1)

def main(i):
    qv = QuantumFloat(i)

    inner_function(qv, 0)
    inner_function(qv, 1)
    inner_function(qv, 2)

    return measure(qv)


t0 = time.time()
jaspr = make_jaspr(main)(5)
print(time.time()- t0)

1.0149471759796143


If a cached function is called with a different type (classical or quantum) the function will not be retrieved from cache but instead retraced. If called with the same signature, the appropriate implementation will be retrieved from the cache.

In [8]:
@qache
def inner_function(qv):
    x(qv)
    time.sleep(1)

def main():
    qf = QuantumFloat(5)
    qbl = QuantumBool(5)

    inner_function(qf)
    inner_function(qf)
    inner_function(qbl)
    inner_function(qbl)

    return measure(qf)

t0 = time.time()
jaspr = make_jaspr(main)()
print(time.time()- t0)

2.033618688583374


We see 2 seconds now because the ``inner_function`` has been traced twice: Once for the ``QuantumFloat`` and once for the ``QuantumBool``.

Another important concept are dynamic values. Dynamic values are values that are only known at runtime (i.e. when the program is actually executed). This could be because the value is coming from a quantum measurement. Every QuantumVariable and it's ``.size`` attribute are dynamic. Furthermore classical values can also be dynamic. For classical values, we can use the Python native ``isinstance`` check for the ``jax.core.Tracer`` class, whether a variable is dynamic. Note that even though ``QuantumVariables`` behave dynamic, they are not tracers themselves.

In [9]:
from jax.core import Tracer

def main(i):
    print("i is dynamic?: ", isinstance(i, Tracer))
    
    qf = QuantumFloat(5)
    j = qf.size
    print("j is dynamic?: ", isinstance(i, Tracer))
    
    h(qf)
    k = measure(qf)
    print("k is dynamic?: ", isinstance(k, Tracer))

    # Regular Python integers are not dynamic
    l = 5
    print("l is dynamic?: ", isinstance(l, Tracer))

    # Arbitrary Python objects can be used within Jasp
    # but they are not dynamic
    import networkx as nx
    G = nx.DiGraph()
    G.add_edge(1,2)
    print("G is dynamic?: ", isinstance(l, Tracer))
    
    return k

jaspr = make_jaspr(main)(5)


i is dynamic?:  True
j is dynamic?:  True
k is dynamic?:  True
l is dynamic?:  False
G is dynamic?:  False


What is the advantage of dynamic values? Dynamical code is scale invariant! For this we can use the ``jrange`` iterator, which allows you to execute a dynamic amount of loop iterations. Some restrictions apply however (check the docs to see which).

In [10]:
@jaspify
def main(k):

    a = QuantumFloat(k)
    b = QuantumFloat(k)

    # Brings a into uniform superposition via Hadamard
    h(a)

    c = measure(a)

    # Excutes c iterations (i.e. depending the measurement outcome)
    for i in jrange(c):

        # Performs a quantum incrementation on b based on the measurement outcome
        b += c//5

    return measure(b)

print(main(5))

0                                                                                    


It is possible to execute a multi-controlled X gate with a dynamic amount of controls.

In [11]:
@jaspify
def main(i, j, k):

    a = QuantumFloat(5)
    a[:] = i
    
    qbl = QuantumBool()

    # a[:j] is a dynamic amount of controls
    mcx(a[:j], qbl[0], ctrl_state = k)

    return measure(qbl)

This function encodes the integer ``i`` into a ``QuantumFloat`` and subsequently performs an MCX gate with control state ``k``. Therefore, we expect the function to return ``True`` if ``i == k`` and ``j > 5``.

In [12]:
print(main(1, 6, 1))
print(main(3, 6, 1))
print(main(2, 1, 1))

True                                                                                 
False                                                                                
False                                                                                


Classical control flow
----------------------

Jasp code can be conditioned on classically known values. For that we simply use the ``control`` feature from base-Qrisp but with dynamical, classical bools. Some restrictions apply (check the docs for more details).

In [13]:
@jaspify
def main():

    qf = QuantumFloat(3)
    h(qf)

    # This is a classical, dynamical int
    meas_res = measure(qf)

    # This is a classical, dynamical bool
    ctrl_bl = meas_res >= 4
    
    with control(ctrl_bl):
        qf -= 4

    return measure(qf)

for i in range(5):
    print(main())

0                                                                                    
0                                                                                    
0                                                                                    
0                                                                                    
3                                                                                    


The RUS decorator
-----------------

RUS stands for Repeat-Until-Success and is an essential part for many quantum algorithms such as HHL or LCU. As the name says the RUS component repeats a certain subroutine until a measurement yields ``True``. The RUS decorator should be applied to a ``trial_function``, which returns a classical bool as the first return value and some arbitrary other values. The trial function will be repeated until the classical bool is ``True``.

To demonstrate the RUS behavior, we initialize a GHZ state 

$\ket{\psi} = \frac{1}{\sqrt{2}} (\ket{00000} + \ket{11111})$

and measure the first qubit into a boolean value. This will be the value to cancel the repetition. This will collapse the GHZ state into either $\ket{00000}$ (which will cause a new repetition) or $\ket{11111} = \ket{31}$, which cancels the loop. After the repetition is canceled we are therefore guaranteed to have the latter state.


In [14]:
from qrisp.jasp import RUS, make_jaspr
from qrisp import QuantumFloat, h, cx, measure

def init_GHZ(qf):
    h(qf[0])
    for i in jrange(1, qf.size):
        cx(qf[0], qf[i])

@RUS
def rus_trial_function():
    qf = QuantumFloat(5)

    init_GHZ(qf)
    
    cancelation_bool = measure(qf[0])
    
    return cancelation_bool, qf

@jaspify
def main():

    qf = rus_trial_function()

    return measure(qf)

print(main())

31.0                                                                                 


Terminal sampling
-----------------

The ``jaspify`` decorator executes one "shot". For many quantum algorithms we however need the distribution of shots. In principle we could execute a bunch of "jaspified" function calls, which is however not as scalable. For this situation we have the ``terminal_sampling`` decorator. To use this decorator we need a function that returns a ``QuantumVariable`` (instead of a classical measurement result). The decorator will then perform a (hybrid) simulation of the given script and subsequently sample from the distribution at the end.

In [15]:

@RUS
def rus_trial_function():
    qf = QuantumFloat(5)

    init_GHZ(qf)
    
    cancelation_bool = measure(qf[0])
    
    return cancelation_bool, qf

@terminal_sampling
def main():

    qf = rus_trial_function()
    h(qf[0])

    return qf

print(main())

{30.0: 0.5, 31.0: 0.5}                                                               


The ``terminal_sampling`` decorator requires some care however. Remember that it only samples from the distribution at the end of the algorithm. This distribution can depend on random chances that happened during the execution. We demonstrate faulty use in the following example.

In [16]:
from qrisp import QuantumBool, measure, control

@terminal_sampling
def main():

    qbl = QuantumBool()
    qf = QuantumFloat(4)

    # Bring qbl into superposition
    h(qbl)

    # Perform a measure
    cl_bl = measure(qbl)

    # Perform a conditional operation based on the measurement outcome
    with control(cl_bl):
        qf[:] = 1
        h(qf[2])

    return qf

for i in range(5):
    print(main())
# Yields either {0.0: 1.0} or {1.0: 0.5, 5.0: 0.5} (with a 50/50 probability)

{0.0: 1.0}                                                                           
{1.0: 0.5, 5.0: 0.5}                                                                 
{1.0: 0.5, 5.0: 0.5}                                                                 
{1.0: 0.5, 5.0: 0.5}                                                                 
{1.0: 0.5, 5.0: 0.5}                                                                 


Boolean simulation
------------------

The tight Jax integration of Jasp enables some powerful features such as a highly performant simulator of purely boolean circuits. This simulator works by transforming Jaspr objects that contain only X, CX, MCX etc. into boolean Jax logic. Subsequently this is inserted into the Jax pipeline, which yields a highly scalable simulator for purely classical Jasp functions.

To call this simulator, we simply use the ``boolean_simulation`` decorator like we did with the ``jaspify`` decorator.

In [17]:
from qrisp import *
from qrisp.jasp import *

def quantum_mult(a, b):
    return a*b

@boolean_simulation(bit_array_padding = 2**10)
def main(i, j, iterations):

    a = QuantumFloat(10)
    b = QuantumFloat(10)

    a[:] = i
    b[:] = j

    c = QuantumFloat(30)

    for i in jrange(iterations): 

        # Compute the quantum product
        temp = quantum_mult(a,b)

        # add into c
        c += temp

        # Uncompute the quantum product
        with invert():
            # The << operator "injects" the quantum variable into
            # the function. This means that the quantum_mult
            # function, which was originally out-of-place, is
            # now an in-place function operating on temp.

            # It can therefore be used for uncomputation
            # Automatic uncomputation is not yet available within Jasp.
            (temp << quantum_mult)(a, b)

        # Delete temp
        temp.delete()

    return measure(c)


The first call needs some time for compilation

In [ ]:
import time
t0 = time.time()
main(1, 2, 5)
print(time.time()-t0)

Any subsequent call is super fast

In [ ]:
t0 = time.time()
print(main(3, 4, 120)) # Expected to be 3*4*120 = 1440
print(f"Took {time.time()-t0} to simulate 120 iterations")

Compile and simulate A MILLION QFLOPs!

In [ ]:
print(main(532, 233, 1000000))

Letting a classical, neural network decide when to stop
-------------------------------------------------------

The following example showcases how a simple neural network can decide (in real-time) whether to go on or break the RUS iteration. For that we create a simple binary classifier and train it on dummy data (disclaimer: ML code by ChatGPT). This is code is not really useful in anyway and the classifier is classifying random data, but it shows how such an algorithm can be constructed and evaluated.

In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit
import optax

# Define the model
def model(params, x):
    W, b = params
    return jax.nn.sigmoid(jnp.dot(x, W) + b)

# Define the loss function (binary cross-entropy)
def loss_fn(params, x, y):
    preds = model(params, x)
    return -jnp.mean(y * jnp.log(preds) + (1 - y) * jnp.log(1 - preds))

# Initialize parameters
key = jax.random.PRNGKey(0)
W = jax.random.normal(key, (2, 1))
b = jax.random.normal(key, (1,))
params = (W, b)

# Create optimizer
optimizer = optax.adam(learning_rate=0.01)
opt_state = optimizer.init(params)

# Define training step
@jit
def train_step(params, opt_state, x, y):
    loss, grads = jax.value_and_grad(loss_fn)(params, x, y)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss

# Generate some dummy data
key = jax.random.PRNGKey(0)
X = jax.random.normal(key, (1000, 2))
y = jnp.sum(X > 0, axis=1) % 2

# Training loop
for epoch in range(100):
    params, opt_state, loss = train_step(params, opt_state, X, y)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

# Make predictions
predictions = model(params, X)
accuracy = jnp.mean((predictions > 0.5) == y)
print(f"Final accuracy: {accuracy}")


We can now use the ``model`` function to evaluate the classifier. Since this function is Jax-based it integrates seamlessly into Jasp.

In [ ]:
from qrisp.jasp import *
from qrisp import *
   
@RUS
def rus_trial_function(params):

    # Sample data from two QuantumFloats.
    # This is a placeholder for an arbitrary quantum algorithm.
    qf_0 = QuantumFloat(5)
    h(qf_0)

    qf_1 = QuantumFloat(5)
    h(qf_1)

    meas_res_0 = measure(qf_0)
    meas_res_1 = measure(qf_1)

    # Turn the data into a Jax array
    X = jnp.array([meas_res_0,meas_res_1])/2**qf_0.size

    # Evaluate the model
    model_res = model(params, X)

    # Determine the cancelation
    cancelation_bool = (model_res > 0.5)[0]
    
    return cancelation_bool, qf_0

@jaspify
def main(params):

    qf = rus_trial_function(params)
    h(qf[0])

    return measure(qf)

print(main(params))